In [0]:
# -*- coding: utf-8 -*-

# Unhuman Project #

## Curiosity - Redy ##

This notebook is about the neural network that can recognize red from an RGB value.

### Getting Started ###

First, we must import the necessary modules:

In [0]:
import tensorflow as tf
import keras
import numpy as np

print("TensorFlow version: {0}\nKeras version: {1}".format(tf.__version__, keras.__version__))

Then we must create the model. It is a Sequential containing 3 dense neuron layers:

In [0]:
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(3,), dtype=tf.float32),
	keras.layers.Dense(units=16, activation="relu", kernel_initializer='glorot_uniform', bias_initializer='zeros'),
	keras.layers.Dense(units=16, activation="relu", kernel_initializer='glorot_uniform', bias_initializer='zeros'),
	keras.layers.Dense(units=16, activation="relu", kernel_initializer='glorot_uniform', bias_initializer='zeros'),
	keras.layers.Dense(units=1, activation="sigmoid", kernel_initializer='glorot_uniform', bias_initializer='zeros')
])

### Training ###

Then, we create a very simple dataset for the training. First of all, we must access Google Drive file to fetch the CSV training file (see [this notebook for more detail about Google Drive integration with Colab Research](https://colab.research.google.com/notebooks/io.ipynb)):

In [0]:
import_file_anyway = False #@param {type:"boolean"}
default_training_filename = "training_dataset (11).csv" #@param {type: "string"}

training_filename = default_training_filename

import os
import os.path
from google.colab import files

raw_content = ""

if not os.path.isfile(training_filename) or import_file_anyway:
    uploaded = {}
    
    while len(uploaded) <= 0:
        uploaded = files.upload()
        print(uploaded)
    
    training_filename = next(iter(uploaded))
    raw_content = uploaded[training_filename].decode('utf-8')
else:
    with open(training_filename, 'r') as f:
        raw_content = f.read()
    
print("Successfully uploaded '{}'.".format(training_filename))

del import_file_anyway
del default_training_filename

Now, let's import the dataset and put them in the lists ```features```,  ```labels``` and  ```dataset```(which contains tuples of feature and label):



In [0]:
import pandas as pd
import io

csv_format = pd.read_csv(io.StringIO(raw_content), skiprows=1)

features = []
labels = []
dataset = []

def convert_color(color):
    if isinstance(color, list):
        if (len(color) == 1 and isinstance(color[0], list)):
            return [[color[0][0] / 255, color[0][1] / 255, color[0][2] / 255]]
    else:
        return [color[0] / 255, color[1] / 255, color[2] / 255]

for index, row in csv_format.iterrows():
    # Change basis [0 ; 255] to [0 ; 1] for features
    features.append(convert_color(row))
    labels.append(row[3])
    dataset.append((features[-1], labels[-1]))

del csv_format

# Print dataset in a human way
print("Number of training entr{1}: {0}".format(len(dataset), 'y' if len(dataset) <= 1 else "ies"))
i = 0
for x, y in dataset:
    print("[{0:.2f}, {1:.2f}, {2:.2f}] -> {3}  ({4})".format(x[0], x[1], x[2], y, "Red-ish" if y > 0.5 else "Not red"))
    i += 1
    if (i >= 15):
        print("... [{} more]".format(len(dataset) - i))
        break

Finally, make the training loop:

In [0]:
import numpy as np

learning_rate = 1. #@param {type: "slider", min:0, max:1, step:0.01}

model.compile(
    optimizer=tf.train.GradientDescentOptimizer(learning_rate), #GradientDescentOptimizer tf.train.AdadeltaOptimizer
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

np_features = np.array(features, dtype=np.float32)
np_labels = np.array(labels, dtype=np.float32)

#print("np_features = ({} containing {} elem) {}\nnp_labels = ({} containing {} elem) {}".format(type(np_features), len(np_features), np_features, type(np_labels), len(np_labels), np_labels))

model.fit(
    x=np_features,
    y=np_labels,
    epochs=10,
    batch_size=32,
    verbose=1,
    validation_data=(np_features, np_labels)
)

### Tests ###

In [0]:
result = model.evaluate(np_features, np_labels, batch_size=32)

print("loss = {0}\naccuracy = {1:.3%}".format(result[0], result[1]))

del result

### Predictions ###

Now, let's make some predictions:

In [0]:
#@title Make some prediction by typing a color: { run: "auto", display-mode: "form" }

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

red   = 243 #@param {type:"slider", min:0, max:255, step:1}
green = 10 #@param {type:"slider", min:0, max:255, step:1}
blue  = 6 #@param {type:"slider", min:0, max:255, step:1}

color = [red, green, blue]
np_color = np.array(convert_color([color]), dtype=np.float32)

# Display the color with matplotlib
try:
    fig, ax = plt.subplots(1)
    rect = matplotlib.patches.Rectangle((0, 0), 1, 1, fill=True, color=(red / 255, green / 255, blue / 255))
    ax.add_patch(rect)
    ax.grid(False)
    plt.axis('off')
    plt.show()
    del fig
    del ax
    del rect
except:
    pass

result = model.predict(np_color)[0][0]
confidence = result if result >= 0.5 else 1. - result
print("Accuracy: {0:.3%}\nThe color is \x1b[1;31m{1}red\x1b[0m ; the program is confident at {2:.2%}".format(result, "" if result > 0.5 else "not ", confidence))

del result
del confidence